In [8]:
import gymnasium as gym
import numpy as np
import pickle

# --- Función de entrenamiento ---
def train_q_learning(
    map_name="4x4",
    episodes=30000,
    learning_rate=0.9,
    discount_factor=0.99,
    epsilon=1.0,
    epsilon_decay=0.0002,
    q_file_name="frozen_lake4x4.pkl"
):
    """
    Entrena un agente Q-learning en el entorno FrozenLake usando el mapa especificado.
    No se visualiza el entorno durante el entrenamiento.

    :param map_name: Nombre del mapa ("4x4" o "8x8", etc.).
    :param episodes: Número de episodios para entrenar.
    :param learning_rate: Tasa de aprendizaje (alpha).
    :param discount_factor: Factor de descuento (gamma).
    :param epsilon: Probabilidad inicial de explorar (política ε-greedy).
    :param epsilon_decay: Tasa a la que disminuye epsilon en cada episodio.
    :param q_file_name: Nombre del archivo donde se guardará la Q-table resultante.
    """
    # Crear el entorno. Sin render, pues no se quiere visualizar nada en entrenamiento.
    env = gym.make('FrozenLake-v1', map_name=map_name, is_slippery=False)

    # Inicializar la Q-table con ceros
    q_table = np.zeros((env.observation_space.n, env.action_space.n))

    # Generador de números aleatorios
    rng = np.random.default_rng()

    for ep in range(episodes):
        # Reiniciamos el entorno y obtenemos el estado inicial
        state = env.reset()[0]
        terminated = False
        truncated = False

        while not terminated and not truncated:
            # Decidimos la acción según la política ε-greedy
            if rng.random() < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(q_table[state, :])

            # Ejecutamos la acción en el entorno
            new_state, reward, terminated, truncated, _ = env.step(action)

            # Actualizamos la Q-table usando la ecuación de Q-learning
            q_table[state, action] += learning_rate * (
                reward
                + discount_factor * np.max(q_table[new_state, :])
                - q_table[state, action]
            )

            # Avanzamos al siguiente estado
            state = new_state

        # Actualización de epsilon (disminuye para reducir la exploración con el tiempo)
        epsilon = max(epsilon - epsilon_decay, 0)

    # Cerramos el entorno tras entrenar
    env.close()

    # Guardamos la Q-table entrenada en un archivo
    with open(q_file_name, "wb") as f:
        pickle.dump(q_table, f)

    print(f"Entrenamiento finalizado. Q-table guardada en: {q_file_name}")

# --- Función de prueba (visualización final) ---
def run_trained_agent(
    map_name="4x4",
    episodes=5,
    q_file_name="frozen_lake4x4.pkl"
):
    """
    Carga una Q-table entrenada y ejecuta varios episodios para
    observar cómo el agente se mueve en FrozenLake (con render activo).

    :param map_name: Nombre del mapa ("4x4" o "8x8", etc.).
    :param episodes: Número de episodios de prueba que se visualizarán.
    :param q_file_name: Archivo pickle donde está almacenada la Q-table entrenada.
    """
    # Creamos el entorno con render para ver la ejecución
    env = gym.make('FrozenLake-v1', map_name=map_name, is_slippery=False, render_mode='human')

    # Cargar la Q-table entrenada
    with open(q_file_name, "rb") as f:
        q_table = pickle.load(f)

    # Para medir cuántos episodios se ganan
    total_wins = 0

    for ep in range(episodes):
        print(f"--- Episodio de prueba {ep+1}/{episodes} ---")
        state = env.reset()[0]
        terminated = False
        truncated = False

        while not terminated and not truncated:
            # Elegimos la acción con el máximo valor Q
            action = np.argmax(q_table[state, :])
            new_state, reward, terminated, truncated, _ = env.step(action)
            state = new_state

            if terminated or truncated:
                # Si reward == 1 significa que llegamos a la meta
                if reward == 1:
                    print("¡El agente ha llegado a la meta!")
                    total_wins += 1
                else:
                    print("El agente cayó en un hoyo o se truncó el episodio.")
                break

    env.close()
    print(f"Episodios ganados: {total_wins}/{episodes} = {100 * total_wins/episodes:.2f}%")

# --- Ejecución principal ---
if __name__ == "__main__":
    # 1) Entrenamos el agente (SIN visualización)
    train_q_learning(
        map_name="4x4",
        episodes=10000,           # Ajusta según quieras más o menos entrenamiento
        learning_rate=0.9,
        discount_factor=0.9,
        epsilon=1.0,
        epsilon_decay=0.0001,
        q_file_name="frozen_lake4x4.pkl"
    )

    # 2) Probamos el agente entrenado (CON visualización)
    run_trained_agent(
        map_name="4x4",
        episodes=5,              # Número de episodios con render
        q_file_name="frozen_lake4x4.pkl"
    )


Entrenamiento finalizado. Q-table guardada en: frozen_lake4x4.pkl
--- Episodio de prueba 1/5 ---
¡El agente ha llegado a la meta!
--- Episodio de prueba 2/5 ---
¡El agente ha llegado a la meta!
--- Episodio de prueba 3/5 ---
¡El agente ha llegado a la meta!
--- Episodio de prueba 4/5 ---
¡El agente ha llegado a la meta!
--- Episodio de prueba 5/5 ---
¡El agente ha llegado a la meta!
Episodios ganados: 5/5 = 100.00%
